In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from xgboost import XGBRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import statsmodels.api as sm        


In [ ]:
df = pd.read_csv("EDA_output.csv")
df.head()

,year,insured_units,insured_persons,claims_total_cnt_k,claims_total_amt_m,claims_inpatient_cnt_k,claims_outpatient_cnt_k,claims_inpatient_amt_m,claims_outpatient_amt_m,total_cost_per_person,...,claims_inpatient_amt_m_lag1,claims_outpatient_amt_m_lag1,insured_persons_lag1,total_cost_per_person_lag1,inpatient_share_amount_lag1,outpatient_share_amount_lag1,gr_insured_persons,gr_claims_total_amt_m,gr_inpatient_amt_m,gr_outpatient_amt_m
0,1997,106082,4057159,38602,47885.0,436,19237.0,38166,28648.0,11802.593884,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1998,110119,4072541,38604,48822.0,395,19023.0,38209,29799.0,11988.092938,...,38166.0,28648.0,4057159.0,11802.593884,0.797035,0.598267,0.003791,0.019568,0.001127,0.040177
2,1999,115306,4108252,37201,40007.0,293,12463.0,36908,27544.0,9738.204959,...,38209.0,29799.0,4072541.0,11988.092938,0.782618,0.610360,0.008769,-0.180554,-0.034050,-0.075674
3,2000,124806,4195952,41775,55529.0,469,21618.0,41307,33911.0,13233.945479,...,36908.0,27544.0,4108252.0,9738.204959,0.922539,0.688480,0.021347,0.387982,0.119188,0.231157
4,2001,125022,4263321,39948,51875.0,435,20172.0,39512,31703.0,12167.744348,...,41307.0,33911.0,4195952.0,13233.945479,0.743882,0.610690,0.016056,-0.065803,-0.043455,-0.065112


In [5]:
feature_cols = [
    # base level
    "insured_persons",
    "inpatient_share_amount",
    "outpatient_share_amount",
    # Avg cost per claim
    "total_cost_per_person",
    "impatient_cost_per_person",
    "outpatient_cost_per_person",
    "avg_cost_per_claim",
    "avg_impatient_cost_per_claim",
    "avg_outpatient_cost_per_claim",
    # lags
    "claims_total_amt_m_lag1",
    "insured_persons_lag1",
    # growths
    "gr_claims_total_amt_m",
    "gr_insured_persons"
]

target_cols = ["claims_total_amt_m", "claims_inpatient_amt_m", "claims_outpatient_amt_m"]

supervised = df[["year"] + feature_cols + target_cols].dropna().reset_index(drop=True)
X = supervised[feature_cols].values
y = supervised[target_cols].values
years = supervised["year"].values

In [6]:
### ----- For Pytorch -------------
### Agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

torch.cuda.manual_seed(42)

in_dim = X.shape[1]
out_dim = y.shape[1]

lr=1e-2
weigh_decay=1e-2

print(f"Input features: {in_dim}")
print(f"Output features: {out_dim}")

Input features: 13
Output features: 3


In [7]:
def Calculate_MAE_RMSE(target_cols, y_true, y_pred):
    results = {}
    for j, name in enumerate(target_cols):
        mae = mean_absolute_error(y_true[:, j], y_pred[:, j])
        rmse = np.sqrt(mean_squared_error(y_true[:, j], y_pred[:, j]))  
        print(f"{name:25s}  MAE = {mae:10.2f},  RMSE = {rmse:10.2f}")
        results[name] = (mae, rmse)
    return results

In [15]:
def rolling_origin_eval_multi(X, y, years, model, min_train_size=8):
    """"
    X: (N, d)
    y: (N, T) multi-output targets
    years: (N,)
    model: regressor supporting multi-output (or wrapped model)
    """
    preds = []
    actuals = []
    pred_years = []
    
    n_samples = X.shape[0]
    
    for split in range(min_train_size, n_samples):
        X_train, y_train = X[:split], y[:split]
        X_test, y_test = X[split].reshape(1, -1), y[split]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)[0]
        
        preds.append(y_pred)
        actuals.append(y_test)
        pred_years.append(years[split])
    
    return model, np.array(preds), np.array(actuals), np.array(pred_years)

In [9]:
class MLPRegressor(nn.Module):
    def __init__(self, input_dim=in_dim, output_dim=out_dim):
        super().__init__()
        
        self.network = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, output_dim)
        )
        
    def forward(self, x):
        return self.network(x)

In [10]:
### ---- For Pytorch training loop  ------- 

def train_torch_mlp(X_train, y_train, n_epochs=200, lr=1e-2, weigh_decay=1e-2, batch_size=None):
    """
    Train an MLPRegressor on X_train, y_train (numpy arrays).
    Returns: trained model, scaler_X, scaler_y
    """
    import torchmetrics
    
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X_train_scaled = scaler_X.fit_transform(X_train)
    y_train_scaled = scaler_y.fit_transform(y_train)
    
    X_tensor = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).to(device)
    
    if batch_size is None or batch_size > X_tensor.shape[0]:
        batch_size = X_tensor.shape[0]
        
    dataset = TensorDataset(X_tensor, y_tensor)
    loader = DataLoader(dataset, 
                        batch_size=batch_size,
                        shuffle=True)
    
    mlp_model = MLPRegressor(input_dim=X_train.shape[1], output_dim=y_train.shape[1]).to(device)
    
    criterion = nn.MSELoss()

    optimizer = torch.optim.Adam(mlp_model.parameters(),
                            lr=lr,
                            weight_decay=weigh_decay)
    # Initialize torchmetrics
    mae_metric = torchmetrics.MeanAbsoluteError().to(device)
    mse_metric = torchmetrics.MeanSquaredError().to(device)
    
    mlp_model.train()
    for epoch in range(n_epochs):
        epoch_loss = 0.0
        mae_metric.reset()
        mse_metric.reset()
        
        for xb, yb in loader:
            optimizer.zero_grad()
            pred = mlp_model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * xb.size(0)
            
            # Update metrics
            mae_metric.update(pred, yb)
            mse_metric.update(pred, yb)
        
        # Calculate average loss for the epoch
        epoch_loss /= X_tensor.shape[0]
        
        if epoch % 10 == 0:
            mae = mae_metric.compute()
            rmse = torch.sqrt(mse_metric.compute())
            print(f"Epoch {epoch:3d}/{n_epochs} - Loss: {epoch_loss:.6f} | MAE: {mae:.6f} | RMSE: {rmse:.6f}")
    
    return mlp_model, scaler_X, scaler_y
    
    

In [16]:
class MLPWrapper:
    def __init__(self, model, scaler_X, scaler_y):
        self.model = model
        self.scaler_X = scaler_X
        self.scaler_y = scaler_y

    def predict(self, X):
        X_scaled = self.scaler_X.transform(X)
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
        self.model.eval()
        with torch.inference_mode():
            y_scaled = self.model(X_tensor).cpu().numpy()
        return self.scaler_y.inverse_transform(y_scaled)

In [17]:
def rolling_origin_eval_torch(X, y, years, min_train_size=8, n_epochs=200, lr=1e-2, weight_decay=1e-2, batch_size=None):
    """"
    X: (N, d)
    y: (N, T) multi-output targets
    years: (N,)
    model: regressor supporting multi-output (or wrapped model)
    """
    preds = []
    actuals = []
    pred_years = []
    
    n_samples = X.shape[0]
    
    for split in range(min_train_size, n_samples):
        X_train, y_train = X[:split], y[:split]
        
        mlp_model, scaler_X, scaler_y = train_torch_mlp(X_train, y_train, 
                                                        n_epochs=n_epochs, 
                                                        lr=lr, 
                                                        weight_decay=weight_decay, 
                                                        batch_size=batch_size)
        
        X_test = X[split].reshape(1, -1)
        X_test_scaled = scaler_X.transform(X_test)
        X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)
        y_test = y[split]
        
        mlp_model.eval()
        with torch.inference_mode():
            y_pred_scaled = mlp_model(X_test_tensor).cpu().numpy()
        
        y_pred = scaler_y.inverse_transform(y_pred_scaled)[0]
    
        preds.append(y_pred)
        actuals.append(y_test)
        pred_years.append(years[split])
        
        # keep last model + scalers
        final_model = mlp_model
        final_scaler_X = scaler_X
        final_scaler_y = scaler_y
        
        mlp_wrapper = MLPWrapper(final_model, final_scaler_X, final_scaler_y)
    
    return mlp_wrapper, np.array(preds), np.array(actuals), np.array(pred_years)

In [12]:
def rolling_origin_arima(y_series, years, order=(1,1,1), min_train_size=8):
    preds, actuals, pred_years = [], [], []
    n = len(y_series)

    for split in range(min_train_size, n):

        # Train ARIMA on y[:split]
        model = sm.tsa.ARIMA(y_series[:split], order=order)
        result = model.fit()

        # Predict next year (t+1)
        pred = result.forecast(steps=1)[0]
        true = y_series[split]

        preds.append(pred)
        actuals.append(true)
        pred_years.append(years[split])

    return np.array(preds), np.array(actuals), np.array(pred_years)

In [14]:
def calculate_mae_rmse_single(name, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print(f"{name:25s}  MAE = {mae:10.2f},  RMSE = {rmse:10.2f}")
    return mae, rmse

In [ ]:
def forecast_model_selec(model_name="xgboost"):
    """
    Select the model and run rolling-origin evaluation, print MAE/RMSE for selected targets.

    Returns
    -------
    trained_model : fitted model or None
        - 'rf' / 'xgboost' : fitted multi-output model
        - 'mlp'           : fitted torch model (as returned by rolling_origin_eval_torch)
        - 'arima'         : None for now (ARIMA serving not wired yet)
    results : dict or DataFrame
        MAE/RMSE per target.
    """
    if model_name == "rf":
        rf_base = RandomForestRegressor(
            n_estimators=500,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            # max_features='auto',
            random_state=42, 
        )
        trained_model, preds, actuals, pred_years = rolling_origin_eval_multi(X, y, years, rf_base, min_train_size=8)
    elif model_name == "xgboost":
        xgb_base = XGBRegressor(
            objective="reg:squarederror",
            n_estimators=300,
            max_depth=3,
            learning_rate=0.05,
            subsample=0.9,
            colsample_bytree=0.9,
            reg_lambda=1.0,
            random_state=42,
        )
        base_model = MultiOutputRegressor(xgb_base)  # MultiOutputRegressor(XGBRegressor) 
        trained_model, preds, actuals, pred_years = rolling_origin_eval_multi(X, y, years, base_model, min_train_size=8)
    elif model_name == "mlp":
        # You may prefer to re-train or store a fitted MLP
        trained_model, preds, actuals, pred_years = rolling_origin_eval_torch(X, y, years)
    elif model_name == "arima":
        # ARIMA-only forecast for total, inpatient, outpatient
        preds, actuals, years = rolling_origin_arima(
            y[:, 0], years, order=(1,1,1)
        )
        mae, rmse = calculate_mae_rmse_single("claims_total_amt_m", actuals, preds)
        results_a = {"claims_total_amt_m": (mae, rmse)}
        return None, results_a
    else:
        raise ValueError(f"Unknown model_name: {model_name}")
    
    results = Calculate_MAE_RMSE(target_cols, actuals, preds) 
    
    return trained_model, results
   

In [18]:
def forecast_claims(horizon=1, model_name=None, trained_model=None):
    """
    Use a trained model to forecast claims_total_amt_m, claims_inpatient_amt_m,
    claims_outpatient_amt_m for the next `horizon` years from the last available year.

    For now: horizon=1 only.

    Parameters
    ----------
    horizon : int
        Number of years ahead to forecast (currently must be 1).
    model_name : str or None
        Optional explicit model name: 'xgboost', 'rf', 'mlp', 'arima'.
        If None, it will be inferred from `trained_model`.
    trained_model :
        - 'xgboost'/'rf'/'multioutput': fitted sklearn regressor with .predict(X)
        - 'mlp'                     : MLPWrapper instance exposing .predict(X)
        - 'arima'                   : dict {target_name: fitted ARIMA result object}
    """

    # --- small helper: infer model_name from the trained_model type ---
    def _infer_model_name(m):
        # ARIMA: dict of results objects
        if isinstance(m, dict):
            return "arima"

        # Try to detect sklearn / xgboost types
        try:
            from sklearn.ensemble import RandomForestRegressor
            from sklearn.multioutput import MultiOutputRegressor
        except ImportError:
            RandomForestRegressor = type("RFPlaceholder", (), {})
            MultiOutputRegressor = type("MOPPlaceholder", (), {})

        try:
            from xgboost import XGBRegressor
        except ImportError:
            XGBRegressor = type("XGBPlaceholder", (), {})

        # RF
        if isinstance(m, RandomForestRegressor):
            return "rf"

        # MultiOutputRegressor(XGBRegressor)
        if isinstance(m, MultiOutputRegressor):
            if isinstance(m.estimator, XGBRegressor):
                return "xgboost"
            # generic multi-output – you can extend this if you add more models
            return "multioutput"

        # (Optional) detect MLP by torch.nn.Module
        try:
            import torch.nn as nn
            if isinstance(m, nn.Module):
                return "mlp"
        except ImportError:
            pass

        return None

    # ----------------- basic checks -----------------
    last_row = supervised.iloc[-1]
    last_year = int(last_row["year"])

    if horizon != 1:
        raise NotImplementedError("forecast_claims currently supports horizon=1 only.")

    if trained_model is None:
        raise ValueError(
            "forecast_claims expected a pre-trained model. "
            "Call forecast_model_selec() first and pass its trained_model here."
        )

    # auto-infer model_name if not provided
    if model_name is None:
        model_name = _infer_model_name(trained_model)
        if model_name is None:
            raise ValueError(
                "Could not infer model_name from trained_model. "
                "Please pass model_name explicitly (e.g. 'xgboost', 'rf', 'arima')."
            )

    # ---------- ARIMA branch ----------
    if model_name == "arima":
        # trained_model is expected to be dict: {col_name: ARIMAResults-like}
        forecasts = {}
        for col in target_cols:
            arima_result = trained_model[col]   # ARIMAResults
            pred = float(arima_result.forecast(steps=1)[-1])
            forecasts[col] = pred

        return {
            "base_year": last_year,
            "forecast_year": last_year + 1,
            "model_name": "arima",
            "forecasts": forecasts,
        }

    # ---------- ML multi-output branch ----------
    X_last = last_row[feature_cols].values.reshape(1, -1)

    if model_name in ["xgboost", "rf", "multioutput", "mlp"]:
        # MultiOutputRegressor(XGBRegressor) or RandomForestRegressor
        y_pred = trained_model.predict(X_last)[0]  # shape: (n_targets,)

    else:
        raise ValueError(f"Unknown or unsupported model_name: {model_name}")

    forecasts = {name: float(val) for name, val in zip(target_cols, y_pred)}

    return {
        "base_year": last_year,
        "forecast_year": last_year + 1,
        "model_name": model_name,
        "forecasts": forecasts,
    }

### Numeric trend summary

This gives the LLM everything it needs to say things like:

“Over the last 5 years, total claims increased by 23% (CAGR ~4.2%/year), with volatility (std) of …”

In [19]:
def summarize_trends(window=5):
    """
    Compute numeric trend summaries over the last `window` years for each target.

    Returns
    -------
    summary_dict : dict
        {
          target_name: {
            'start_year': int,
            'end_year': int,
            'start_value': float,
            'end_value': float,
            'abs_change': float,
            'pct_change': float,   # in %
            'cagr': float,         # in % per year
            'mean': float,
            'std': float,
          },
          ...
        }
    summary_df : pd.DataFrame
        Tabular version of the same information.
    """
    # Aggregate to yearly sums (or use .mean() if that makes more sense for you)
    yearly = (
        supervised
        .groupby("year")[target_cols]
        .sum()
        .sort_index()
    )

    if yearly.shape[0] < 2:
        raise ValueError("Not enough years of data to summarize trends.")

    # restrict to last `window` years if available
    if yearly.shape[0] > window:
        yearly_window = yearly.iloc[-window:]
    else:
        yearly_window = yearly

    years_idx = yearly_window.index.to_numpy()
    start_year = int(years_idx[0])
    end_year = int(years_idx[-1])
    n_periods = len(years_idx) - 1  # for CAGR

    summary_rows = []

    for col in target_cols:
        series = yearly_window[col].astype(float)
        start_val = float(series.iloc[0])
        end_val = float(series.iloc[-1])
        abs_change = end_val - start_val

        if start_val != 0:
            pct_change = (abs_change / start_val) * 100.0
        else:
            pct_change = float("nan")

        # CAGR: (end/start)^(1/years) - 1
        if start_val > 0 and n_periods > 0:
            cagr = ((end_val / start_val) ** (1.0 / n_periods) - 1.0) * 100.0
        else:
            cagr = float("nan")

        mean_val = float(series.mean())
        std_val = float(series.std(ddof=1)) if len(series) > 1 else float("nan")

        row = {
            "target": col,
            "start_year": start_year,
            "end_year": end_year,
            "start_value": start_val,
            "end_value": end_val,
            "abs_change": abs_change,
            "pct_change": pct_change,
            "cagr": cagr,
            "mean": mean_val,
            "std": std_val,
        }
        summary_rows.append(row)

    summary_df = pd.DataFrame(summary_rows).set_index("target")
    summary_dict = {row["target"]: {k: v for k, v in row.items() if k != "target"}
                    for row in summary_rows}

    return summary_dict, summary_df

### Feature shocks + Impact

Goal: “If I change some drivers (insured people, cost per person, etc.) by X%, what happens to next year’s claims?”

In [20]:
def simulate_policy_change(trained_model, model_name=None, shocks=None, horizon=1):
    """
    Apply multiplicative shocks to selected features and compute impact on forecasted claims.

    Parameters
    ----------
    trained_model :
        Fitted model with .predict(X) interface (RF/XGB/MultiOutputRegressor/MLPWrapper).
    model_name : str or None
        'xgboost', 'rf', 'mlp', 'multioutput', 'arima', or None to infer.
    shocks : dict or None
        Mapping feature_name -> multiplicative factor.
        Example: {'insured_persons': 1.10, 'avg_cost_per_person': 0.95}
        If None or empty dict, no changes applied.
    horizon : int
        Currently must be 1 (same limitation as forecast_claims).

    Returns
    -------
    result : dict
        {
          'base_year': int,
          'forecast_year': int,
          'model_name': str,
          'baseline_forecasts': {target: float},
          'scenario_forecasts': {target: float},
          'impact': {
             target: {
                'abs_change': float,
                'pct_change': float
             },
             ...
          },
          'applied_shocks': shocks_dict actually used,
        }
    """

    if shocks is None:
        shocks = {}

    # --- helper: reuse the same infer logic used in forecast_claims ---
    def _infer_model_name(m):
        if isinstance(m, dict):
            return "arima"

        try:
            from sklearn.ensemble import RandomForestRegressor
            from sklearn.multioutput import MultiOutputRegressor
        except ImportError:
            RandomForestRegressor = type("RFPlaceholder", (), {})
            MultiOutputRegressor = type("MOPPlaceholder", (), {})

        try:
            from xgboost import XGBRegressor
        except ImportError:
            XGBRegressor = type("XGBPlaceholder", (), {})

        if isinstance(m, RandomForestRegressor):
            return "rf"

        if isinstance(m, MultiOutputRegressor):
            if isinstance(m.estimator, XGBRegressor):
                return "xgboost"
            return "multioutput"

        # MLPWrapper
        try:
            if isinstance(m, MLPWrapper):
                return "mlp"
        except NameError:
            pass

        try:
            import torch.nn as nn
            if isinstance(m, nn.Module):
                return "mlp"
        except ImportError:
            pass

        return None

    if trained_model is None:
        raise ValueError("simulate_policy_change requires a trained_model.")

    if horizon != 1:
        raise NotImplementedError("simulate_policy_change currently supports horizon=1 only.")

    # infer model_name if needed
    if model_name is None:
        model_name = _infer_model_name(trained_model)
        if model_name is None:
            raise ValueError(
                "Could not infer model_name from trained_model. "
                "Please pass model_name explicitly (e.g. 'xgboost', 'rf', 'mlp')."
            )

    if model_name == "arima":
        raise NotImplementedError(
            "simulate_policy_change is not implemented for ARIMA-only models, "
            "since it relies on feature shocks."
        )

    # ---------- baseline forecast using existing tool ----------
    baseline = forecast_claims(
        horizon=horizon,
        model_name=model_name,
        trained_model=trained_model,
    )
    baseline_forecasts = baseline["forecasts"]
    base_year = baseline["base_year"]
    forecast_year = baseline["forecast_year"]

    # ---------- build scenario features ----------
    last_row = supervised.iloc[-1].copy()

    applied_shocks = {}

    for feat, factor in shocks.items():
        if feat in last_row.index:
            original_val = last_row[feat]
            last_row[feat] = original_val * factor
            applied_shocks[feat] = {
                "original": float(original_val),
                "factor": float(factor),
                "new_value": float(last_row[feat]),
            }
        else:
            # feature name not found; you could also warn/log instead of ignoring.
            # For now just skip it.
            continue

    X_scenario = last_row[feature_cols].values.reshape(1, -1)

    # ---------- scenario prediction ----------
    # trained_model implements .predict for all supported model_name values
    y_pred_scenario = trained_model.predict(X_scenario)[0]  # (n_targets,)

    scenario_forecasts = {
        name: float(val) for name, val in zip(target_cols, y_pred_scenario)
    }

    # ---------- impact calculation ----------
    impact = {}
    for tgt in target_cols:
        base_val = float(baseline_forecasts[tgt])
        scen_val = float(scenario_forecasts[tgt])
        abs_change = scen_val - base_val
        if base_val != 0:
            pct_change = (abs_change / base_val) * 100.0
        else:
            pct_change = float("nan")
        impact[tgt] = {
            "abs_change": abs_change,
            "pct_change": pct_change,
        }

    result = {
        "base_year": base_year,
        "forecast_year": forecast_year,
        "model_name": model_name,
        "baseline_forecasts": baseline_forecasts,
        "scenario_forecasts": scenario_forecasts,
        "impact": impact,
        "applied_shocks": applied_shocks,
    }

    return result